#### **KẾT NỐI GOOGLE DRIVE VÀ IMPORT THƯ VIỆN**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

#### **THÔNG TIN VỀ CAFEF.VN**

CafeF.vn (hay Kênh thông tin Kinh tế - Tài chính Việt Nam) là một trang tin tức chuyên cung cấp thông tin tài chính, chứng khoán, kinh tế và các tin tức liên quan đến thị trường. Đây là nguồn thông tin phổ biến dành cho nhà đầu tư, doanh nghiệp và những ai quan tâm đến các lĩnh vực kinh tế trong nước và quốc tế. Toàn bộ tin tức của Cafef.vn được lưu trữ tại đường dẫn:
```
https://cafef.vn/timelinelist/{ID}/{Page}.chn
```
Dưới đây là các nội dung chính trên CafeF.vn:

|ID   |Field                 |Max Page|
|-----|----------------------|--------|
|18831|Thị Trường Chứng Khoán|4942    |
|18835|Bất động sản          |5296    |
|18836|Doanh nghiệp          |6034    |
|18834|Tài chính - Ngân hàng |4405    |
|18832|Tài chính quốc tế     |5537    |
|18833|Vĩ mô - Đầu tư        |5494    |
|18839|Thị trường            |3941    |

#### **XÂY DỰNG HÀM CRAWL TIN TỨC**

In [ ]:
class StockScraper:
    def __init__(self, field, start, end):
        self.base_url = 'https://cafef.vn'
        self.field = field
        self.start = start
        self.end = end

    def get_top_stock(self):
        try:
            res = requests.get(f'{self.base_url}/get-top-stock.chn')
            data = res.json()['stocks']
        except:
            return "Không thể tìm nạp dữ liệu"
        return data

    def get_info_stock(self, stock_name):
        try:
            res = requests.get(f'{self.base_url}/info.ashx?type=cp&symbol={stock_name}')
            data = res.json()
        except:
            return "Không thể tìm nạp dữ liệu hoặc mã cổ phiếu cần tìm bị sai"
        return data

    def get_list_post(self, page_num):
        try:
            res = requests.get(f'{self.base_url}/timelinelist/{self.field}/{page_num}.chn')
            soup = BeautifulSoup(res.content, 'html.parser')
            data = soup.find_all(class_='tlitem box-category-item')
        except Exception as e:
            return f"gặp lỗi: {e}"
        return data

    def get_info_post(self, data):
        times = []
        links = []
        titles = []
        for i in range(len(data)):
          time = data[i].find(class_='time time-ago').text
          time_object = datetime.fromisoformat(time)
          if time_object.date() >= self.start and time_object.date() <= self.end:
            times.append(time_object.date())
            links.append(self.base_url + data[i].find('a', href=True)['href'])
            titles.append(data[i].find('a', href=True).text)
        return times, links, titles

    def get_content_post(self, link_post):
        content = ''
        try:
            res = requests.get(link_post)
            soup = BeautifulSoup(res.content, 'html.parser')
            c = soup.find(class_='detail-content afcbc-body')
            p = c.find_all('p')
            for i in range(len(p)):
                content = content + p[i].text + '\n'
        except Exception as e:
            return f"gặp lỗi: {e}"
        return content

    def scrape(self, num_pages_start, num_pages_end):
        df = pd.DataFrame()
        current_year = datetime(2024, 1, 1).date().year
        pages = []
        for i in range(num_pages_start, num_pages_end+1):
            if i%50 == 0:
                print(f'Extracted Page {i}')
            times, links, titles = self.get_info_post(self.get_list_post(i))
            temp_df = pd.DataFrame({'Title': titles, 'Link': links, 'Time': times})
            df = pd.concat([df, temp_df]).reset_index(drop=True)
            if times[0].year != current_year:
                pages.append([current_year, i])
                current_year = times[0].year
        return df, pages

In [ ]:
for i, j in [[18831, 4942], [18835, 5296], [18836, 6034], [18834, 4405], [18832, 5537], [18833, 5494], [18839, 3941]]:

    scraper = StockScraper(i, datetime(2000, 1, 1).date(), datetime(2024, 12, 30).date())
    df, pages = scraper.scrape(1, j)

    df.to_csv(f'/Data/News Data/{i}_all_data.csv')
    pd.DataFrame(pages, columns=['Year', 'Page']).to_csv(f'/Data/News Data/{i}_pages.csv', index=False)

    print(f'Finished extracting branch {i}')

---